# Expected losses and VaR

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaarutyunov/credit-risk/blob/master/notebooks/colab_el_var.ipynb)

## Environment settings

For better performance change Colab runtime type to GPU

In [1]:
import numpy as np
import scipy.stats
!git clone https://github.com/gaarutyunov/credit-risk.git

Cloning into 'credit-risk'...
remote: Enumerating objects: 431, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 431 (delta 50), reused 74 (delta 30), pack-reused 334
Receiving objects: 100% (431/431), 10.94 MiB | 15.34 MiB/s, done.
Resolving deltas: 100% (254/254), done.


In [2]:
%cd credit-risk

/content/credit-risk


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/sberbank-ai/wing.git (to revision master) to /tmp/pip-install-4dhu6sqg/wing_83adb08c7aed4405b07c4f7de6ed9eed
  Running command git clone -q https://github.com/sberbank-ai/wing.git /tmp/pip-install-4dhu6sqg/wing_83adb08c7aed4405b07c4f7de6ed9eed
     |████████████████████████████████| 151 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 76.6 MB 102 kB/s 
     |████████████████████████████████| 117 kB 54.1 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
  Created wheel for wing: filename=wing-0.1.6-py3-none-any.whl size=12768 sha256=41b49e4684f11dcc3e79517dfa923c66d56b12211164dc38c6ec4741edda11ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-ucme3aom/wheels/d0/d3/78/d1cb5717341968f8319fa57765f68a894fc1b21d4f4068b519
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_r

To get username and key follow instructions in [readme](https://github.com/Kaggle/kaggle-api)

In [ ]:
%env KAGGLE_USERNAME=<username>
%env KAGGLE_KEY=<key>

In [5]:
!kaggle datasets download wordsforthewise/lending-club

 99% 1.25G/1.26G [00:08<00:00, 114MB/s]
100% 1.26G/1.26G [00:08<00:00, 164MB/s]


In [6]:
!unzip lending-club.zip

Archive:  lending-club.zip
  inflating: accepted_2007_to_2018Q4.csv.gz  
  inflating: accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv  
  inflating: rejected_2007_to_2018Q4.csv.gz  
  inflating: rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv  


In [7]:
!mkdir data

In [8]:
!mv accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv data/accepted_2007_to_2018Q4.csv
!mv rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv data/rejected_2007_to_2018Q4.csv

## Preprocessing

In [10]:
from pipeline import get_pipeline

preprocessing = get_pipeline(
    name="cat_boost",
    group="preprocessing",
    overrides=[
        "preprocessing_pipeline=raw_data"
    ],
    debug=True,
)

_target_: pipeline.ReaderPipeline
memory: ./cache/preprocessing/raw
steps:
- - CSVReader
  - _target_: pipeline.CSVReader
    _convert_: all
    file: data/accepted_2007_to_2018Q4.csv
    columns:
    - loan_amnt
    - term
    - emp_title
    - emp_length
    - home_ownership
    - verification_status
    - purpose
    - zip_code
    - addr_state
    - earliest_cr_line
    - fico_range_low
    - fico_range_high
    - revol_bal
    - application_type
    - verification_status_joint
    - sec_app_earliest_cr_line
    - loan_status
    - issue_d
    - funded_amnt
    - disbursement_method
- - EmpTitle
  - _target_: pipeline.JobTransformer
    _convert_: all
    max_jobs: 20
- - ImputeNumerical
  - _target_: pipeline.ApplyToColumns
    _convert_: all
    inner:
      _target_: sklearn.impute.SimpleImputer
      strategy: mean
    columns:
    - loan_amnt
    - fico_range_low
    - fico_range_high
    - revol_bal
- - ImputeCategorical
  - _target_: pipeline.ApplyToColumns
    _convert_: al

In [11]:
X = preprocessing.fit_transform([], y=[])

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:355: UserWarning: Persisting input arguments took 118.02s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name],
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:355: UserWarning: Persisting input arguments took 119.18s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name],


In [12]:
import pandas as pd

X['issue_d'] = pd.to_datetime(X['issue_d'])

In [13]:
X = X[X['issue_d'] >= '01.01.2017']

In [14]:
X['issue_d'] = X['issue_d'].dt.strftime('%b-%Y')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
import numpy as np
import scipy.stats


def vasicek(PD, rho, alpha):
    return ( scipy.stats.norm.ppf(PD) + np.sqrt(rho) * scipy.stats.norm.ppf(alpha) ) / np.sqrt(1 - rho)


def mean_confidence_interval(data, confidence=0.999):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

## Cat Boost

In [16]:
from pipeline import get_pipeline

catboost = get_pipeline(
    name="cat_boost",
    group="prediction",
    debug=True,
)

_target_: sklearn.pipeline.Pipeline
memory: ./cache/prediction/cat_boost
steps:
- - Predictor
  - _target_: pipeline.CatBoostLoader
    _convert_: all
    load: models/cat_boost



In [18]:
X["PD_CB"] = catboost.predict_proba(X.drop(columns=["funded_amnt", "issue_d", "loan_status"]))[:, 1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


421097      True
421098      True
421099     False
421100     False
421101      True
           ...  
2157146    False
2157147    False
2157148     True
2157149    False
2157150    False
Name: term, Length: 938821, dtype: bool

In [19]:
LGD = 1.0

In [20]:
X["EL_CB"] = LGD * X["PD_CB"] * X["funded_amnt"]

print(f"Expected losses: {X['EL_CB'].sum():.2f}")

Expected losses: 3167199365.80


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
term_36_mask = X.term.str.strip().str.startswith('36')

In [42]:
VaR_1 = vasicek(X.loc[term_36_mask, "PD_CB"], 0, .999) * X.loc[term_36_mask, "funded_amnt"]
VaR_6_1 = vasicek(X.loc[term_36_mask, "PD_CB"], .06, .999) * X.loc[term_36_mask, "funded_amnt"]

In [63]:
VaR = vasicek(X["PD_CB"], 0, .999) * X["funded_amnt"]
-VaR.sum()

12079836133.868872

In [64]:
mean_confidence_interval(VaR)

34.16002142350049

In [65]:
X["EL_CB"].sum()

3167199365.797802

In [66]:
C = -VaR.sum() - X["EL_CB"].sum()
C

8912636768.07107

In [67]:
VaR_6 = vasicek(X["PD_CB"], 0.06, .999) * X["funded_amnt"]
-VaR_6.sum()

1122196230.4585857

In [68]:
mean_confidence_interval(VaR_6)

24.04852008398609

In [69]:
C = -VaR_6.sum() - X["EL_CB"].sum()
C

-2045003135.3392162